In [40]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [41]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [42]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [43]:
covid_data = pd.read_csv('counties.csv')
covid_df = pd.DataFrame(covid_data)
covid_df.head()

,fips,country,state,county,level,lat,locationId,long,population,metrics.testPositivityRatio,...,actuals.vaccinesDistributed,actuals.vaccinationsInitiated,actuals.vaccinationsCompleted,lastUpdatedDate,url,metrics.vaccinationsInitiatedRatio,metrics.vaccinationsCompletedRatio,actuals.newDeaths,actuals.vaccinesAdministered,cdcTransmissionLevel
0,2013,US,AK,Aleutians East Borough,county,NaN,iso1:us#iso2:us-ak#fips:02013,NaN,3337,NaN,...,NaN,2512.0,2313.0,2021-11-04,https://covidactnow.org/us/alaska-ak/county/al...,0.753,0.693,0.0,NaN,2
1,2016,US,AK,Aleutians West Census Area,county,NaN,iso1:us#iso2:us-ak#fips:02016,NaN,5634,0.000,...,NaN,3738.0,3152.0,2021-11-04,https://covidactnow.org/us/alaska-ak/county/al...,0.663,0.559,0.0,NaN,1
2,2020,US,AK,Anchorage Municipality,county,NaN,iso1:us#iso2:us-ak#fips:02020,NaN,288000,0.083,...,NaN,185037.0,162085.0,2021-11-04,https://covidactnow.org/us/alaska-ak/county/an...,0.642,0.563,5.0,NaN,3
3,2050,US,AK,Bethel Census Area,county,NaN,iso1:us#iso2:us-ak#fips:02050,NaN,18386,0.006,...,NaN,11597.0,10675.0,2021-11-04,https://covidactnow.org/us/alaska-ak/county/be...,0.631,0.581,0.0,NaN,3
4,2060,US,AK,Bristol Bay Borough,county,NaN,iso1:us#iso2:us-ak#fips:02060,NaN,836,NaN,...,NaN,NaN,NaN,2021-11-04,https://covidactnow.org/us/alaska-ak/county/br...,NaN,NaN,0.0,NaN,0


In [44]:
column_list = []
for column in covid_df.columns:
    column_list.append(column)
column_list

['fips',
 'country',
 'state',
 'county',
 'level',
 'lat',
 'locationId',
 'long',
 'population',
 'metrics.testPositivityRatio',
 'metrics.testPositivityRatioDetails.source',
 'metrics.caseDensity',
 'metrics.contactTracerCapacityRatio',
 'metrics.infectionRate',
 'metrics.infectionRateCI90',
 'unused1',
 'unused2',
 'metrics.icuCapacityRatio',
 'riskLevels.overall',
 'riskLevels.testPositivityRatio',
 'riskLevels.caseDensity',
 'riskLevels.contactTracerCapacityRatio',
 'riskLevels.infectionRate',
 'unused3',
 'riskLevels.icuCapacityRatio',
 'actuals.cases',
 'actuals.deaths',
 'actuals.positiveTests',
 'actuals.negativeTests',
 'actuals.contactTracers',
 'actuals.hospitalBeds.capacity',
 'actuals.hospitalBeds.currentUsageTotal',
 'actuals.hospitalBeds.currentUsageCovid',
 'unused4',
 'actuals.icuBeds.capacity',
 'actuals.icuBeds.currentUsageTotal',
 'actuals.icuBeds.currentUsageCovid',
 'unused5',
 'actuals.newCases',
 'actuals.vaccinesDistributed',
 'actuals.vaccinationsInitiated',

In [45]:
drop_list = ['level','lat','locationId','long','unused1','unused2','unused3','unused4','unused5','url']
covid_df.drop(drop_list, axis=1, inplace=True)
covid_df.head()

,fips,country,state,county,population,metrics.testPositivityRatio,metrics.testPositivityRatioDetails.source,metrics.caseDensity,metrics.contactTracerCapacityRatio,metrics.infectionRate,...,actuals.newCases,actuals.vaccinesDistributed,actuals.vaccinationsInitiated,actuals.vaccinationsCompleted,lastUpdatedDate,metrics.vaccinationsInitiatedRatio,metrics.vaccinationsCompletedRatio,actuals.newDeaths,actuals.vaccinesAdministered,cdcTransmissionLevel
0,2013,US,AK,Aleutians East Borough,3337,NaN,other,12.8,NaN,1.00,...,0,NaN,2512.0,2313.0,2021-11-04,0.753,0.693,0.0,NaN,2
1,2016,US,AK,Aleutians West Census Area,5634,0.000,other,5.1,NaN,1.00,...,2,NaN,3738.0,3152.0,2021-11-04,0.663,0.559,0.0,NaN,1
2,2020,US,AK,Anchorage Municipality,288000,0.083,other,79.2,NaN,0.88,...,172,NaN,185037.0,162085.0,2021-11-04,0.642,0.563,5.0,NaN,3
3,2050,US,AK,Bethel Census Area,18386,0.006,other,106.4,NaN,0.83,...,1,NaN,11597.0,10675.0,2021-11-04,0.631,0.581,0.0,NaN,3
4,2060,US,AK,Bristol Bay Borough,836,NaN,other,0.0,NaN,1.00,...,0,NaN,NaN,NaN,2021-11-04,NaN,NaN,0.0,NaN,0


In [46]:

for x in covid_df.columns:
    print(f'{x}: {covid_df[x].nunique()}')

fips: 3222
country: 1
state: 53
county: 1957
population: 3166
metrics.testPositivityRatio: 338
metrics.testPositivityRatioDetails.source: 1
metrics.caseDensity: 816
metrics.contactTracerCapacityRatio: 1
metrics.infectionRate: 92
metrics.infectionRateCI90: 118
metrics.icuCapacityRatio: 103
riskLevels.overall: 5
riskLevels.testPositivityRatio: 5
riskLevels.caseDensity: 5
riskLevels.contactTracerCapacityRatio: 2
riskLevels.infectionRate: 5
riskLevels.icuCapacityRatio: 5
actuals.cases: 2887
actuals.deaths: 654
actuals.positiveTests: 1
actuals.negativeTests: 1
actuals.contactTracers: 1
actuals.hospitalBeds.capacity: 637
actuals.hospitalBeds.currentUsageTotal: 563
actuals.hospitalBeds.currentUsageCovid: 171
actuals.icuBeds.capacity: 245
actuals.icuBeds.currentUsageTotal: 211
actuals.icuBeds.currentUsageCovid: 80
actuals.newCases: 234
actuals.vaccinesDistributed: 2
actuals.vaccinationsInitiated: 3081
actuals.vaccinationsCompleted: 3076
lastUpdatedDate: 1
metrics.vaccinationsInitiatedRatio: 58

In [47]:
covid_df

,fips,country,state,county,population,metrics.testPositivityRatio,metrics.testPositivityRatioDetails.source,metrics.caseDensity,metrics.contactTracerCapacityRatio,metrics.infectionRate,...,actuals.newCases,actuals.vaccinesDistributed,actuals.vaccinationsInitiated,actuals.vaccinationsCompleted,lastUpdatedDate,metrics.vaccinationsInitiatedRatio,metrics.vaccinationsCompletedRatio,actuals.newDeaths,actuals.vaccinesAdministered,cdcTransmissionLevel
0,2013,US,AK,Aleutians East Borough,3337,NaN,other,12.8,NaN,1.00,...,0,NaN,2512.0,2313.0,2021-11-04,0.753,0.693,0.0,NaN,2
1,2016,US,AK,Aleutians West Census Area,5634,0.000,other,5.1,NaN,1.00,...,2,NaN,3738.0,3152.0,2021-11-04,0.663,0.559,0.0,NaN,1
2,2020,US,AK,Anchorage Municipality,288000,0.083,other,79.2,NaN,0.88,...,172,NaN,185037.0,162085.0,2021-11-04,0.642,0.563,5.0,NaN,3
3,2050,US,AK,Bethel Census Area,18386,0.006,other,106.4,NaN,0.83,...,1,NaN,11597.0,10675.0,2021-11-04,0.631,0.581,0.0,NaN,3
4,2060,US,AK,Bristol Bay Borough,836,NaN,other,0.0,NaN,1.00,...,0,NaN,NaN,NaN,2021-11-04,NaN,NaN,0.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,56037,US,WY,Sweetwater County,42343,0.132,other,50.9,NaN,0.98,...,15,NaN,20879.0,16940.0,2021-11-04,0.493,0.370,11.0,NaN,3
3218,56039,US,WY,Teton County,23464,0.060,other,34.7,NaN,0.86,...,3,NaN,21728.0,19110.0,2021-11-04,0.926,0.770,0.0,NaN,3
3219,56041,US,WY,Uinta County,20226,0.119,other,33.9,NaN,0.79,...,10,NaN,9931.0,8552.0,2021-11-04,0.491,0.390,1.0,NaN,3
3220,56043,US,WY,Washakie County,7805,0.048,other,102.5,NaN,0.71,...,11,NaN,3208.0,2898.0,2021-11-04,0.411,0.370,0.0,NaN,3


In [48]:
bed_occupied_rate = covid_df['actuals.hospitalBeds.currentUsageTotal']/covid_df['actuals.hospitalBeds.capacity']
covid_df['Occupied Rate']=bed_occupied_rate
covid_df

,fips,country,state,county,population,metrics.testPositivityRatio,metrics.testPositivityRatioDetails.source,metrics.caseDensity,metrics.contactTracerCapacityRatio,metrics.infectionRate,...,actuals.vaccinesDistributed,actuals.vaccinationsInitiated,actuals.vaccinationsCompleted,lastUpdatedDate,metrics.vaccinationsInitiatedRatio,metrics.vaccinationsCompletedRatio,actuals.newDeaths,actuals.vaccinesAdministered,cdcTransmissionLevel,Occupied Rate
0,2013,US,AK,Aleutians East Borough,3337,NaN,other,12.8,NaN,1.00,...,NaN,2512.0,2313.0,2021-11-04,0.753,0.693,0.0,NaN,2,NaN
1,2016,US,AK,Aleutians West Census Area,5634,0.000,other,5.1,NaN,1.00,...,NaN,3738.0,3152.0,2021-11-04,0.663,0.559,0.0,NaN,1,NaN
2,2020,US,AK,Anchorage Municipality,288000,0.083,other,79.2,NaN,0.88,...,NaN,185037.0,162085.0,2021-11-04,0.642,0.563,5.0,NaN,3,0.916519
3,2050,US,AK,Bethel Census Area,18386,0.006,other,106.4,NaN,0.83,...,NaN,11597.0,10675.0,2021-11-04,0.631,0.581,0.0,NaN,3,NaN
4,2060,US,AK,Bristol Bay Borough,836,NaN,other,0.0,NaN,1.00,...,NaN,NaN,NaN,2021-11-04,NaN,NaN,0.0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,56037,US,WY,Sweetwater County,42343,0.132,other,50.9,NaN,0.98,...,NaN,20879.0,16940.0,2021-11-04,0.493,0.370,11.0,NaN,3,0.607843
3218,56039,US,WY,Teton County,23464,0.060,other,34.7,NaN,0.86,...,NaN,21728.0,19110.0,2021-11-04,0.926,0.770,0.0,NaN,3,0.583333
3219,56041,US,WY,Uinta County,20226,0.119,other,33.9,NaN,0.79,...,NaN,9931.0,8552.0,2021-11-04,0.491,0.390,1.0,NaN,3,0.113636
3220,56043,US,WY,Washakie County,7805,0.048,other,102.5,NaN,0.71,...,NaN,3208.0,2898.0,2021-11-04,0.411,0.370,0.0,NaN,3,0.437500


In [49]:
capacity_clean=[x for x in covid_df['Occupied Rate'] if pd.isnull(x) == False]
capacity_clean

[0.91651865008881,
 0.8765432098765432,
 0.3157894736842105,
 0.9259259259259259,
 0.36,
 0.28,
 0.63125,
 0.25,
 0.5,
 0.6404494382022472,
 0.7150537634408602,
 0.5714285714285714,
 0.36666666666666664,
 0.35714285714285715,
 0.296875,
 0.3448275862068966,
 0.8654545454545455,
 0.52,
 0.24444444444444444,
 0.2,
 0.14285714285714285,
 0.2717391304347826,
 0.2926829268292683,
 0.5342465753424658,
 0.5583941605839416,
 0.5357142857142857,
 0.59375,
 0.40625,
 0.6583850931677019,
 0.4166666666666667,
 0.411214953271028,
 0.65625,
 0.19736842105263158,
 0.3137254901960784,
 0.7521865889212828,
 0.875,
 0.41509433962264153,
 0.5263157894736842,
 0.5,
 0.2857142857142857,
 0.9028006589785832,
 0.5423728813559322,
 0.8345342941019482,
 0.7982456140350878,
 0.09375,
 0.8478260869565217,
 0.5859375,
 0.8366412213740458,
 0.6666666666666666,
 0.36764705882352944,
 0.6190476190476191,
 0.8,
 0.1836734693877551,
 0.7371188222923238,
 0.7098039215686275,
 0.38,
 0.4375,
 0.6923076923076923,
 0.8409

In [59]:
covid_df.loc[covid_df['Occupied Rate']>=0.8,'status']='Full'
covid_df.loc[covid_df['Occupied Rate']<0.8,'status']='Spacious'
covid_df

,fips,country,state,county,population,metrics.testPositivityRatio,metrics.testPositivityRatioDetails.source,metrics.caseDensity,metrics.contactTracerCapacityRatio,metrics.infectionRate,...,actuals.vaccinationsInitiated,actuals.vaccinationsCompleted,lastUpdatedDate,metrics.vaccinationsInitiatedRatio,metrics.vaccinationsCompletedRatio,actuals.newDeaths,actuals.vaccinesAdministered,cdcTransmissionLevel,Occupied Rate,status
0,2013,US,AK,Aleutians East Borough,3337,NaN,other,12.8,NaN,1.00,...,2512.0,2313.0,2021-11-04,0.753,0.693,0.0,NaN,2,NaN,NaN
1,2016,US,AK,Aleutians West Census Area,5634,0.000,other,5.1,NaN,1.00,...,3738.0,3152.0,2021-11-04,0.663,0.559,0.0,NaN,1,NaN,NaN
2,2020,US,AK,Anchorage Municipality,288000,0.083,other,79.2,NaN,0.88,...,185037.0,162085.0,2021-11-04,0.642,0.563,5.0,NaN,3,0.916519,Full
3,2050,US,AK,Bethel Census Area,18386,0.006,other,106.4,NaN,0.83,...,11597.0,10675.0,2021-11-04,0.631,0.581,0.0,NaN,3,NaN,NaN
4,2060,US,AK,Bristol Bay Borough,836,NaN,other,0.0,NaN,1.00,...,NaN,NaN,2021-11-04,NaN,NaN,0.0,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,56037,US,WY,Sweetwater County,42343,0.132,other,50.9,NaN,0.98,...,20879.0,16940.0,2021-11-04,0.493,0.370,11.0,NaN,3,0.607843,Spacious
3218,56039,US,WY,Teton County,23464,0.060,other,34.7,NaN,0.86,...,21728.0,19110.0,2021-11-04,0.926,0.770,0.0,NaN,3,0.583333,Spacious
3219,56041,US,WY,Uinta County,20226,0.119,other,33.9,NaN,0.79,...,9931.0,8552.0,2021-11-04,0.491,0.390,1.0,NaN,3,0.113636,Spacious
3220,56043,US,WY,Washakie County,7805,0.048,other,102.5,NaN,0.71,...,3208.0,2898.0,2021-11-04,0.411,0.370,0.0,NaN,3,0.437500,Spacious


In [62]:
status_df = [x for x in covid_df['status'] if pd.isnull(x) == False]
status_df

['Full',
 'Full',
 'Spacious',
 'Full',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Full',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Full',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Full',
 'Spacious',
 'Full',
 'Spacious',
 'Spacious',
 'Full',
 'Spacious',
 'Full',
 'Spacious',
 'Spacious',
 'Spacious',
 'Full',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Full',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Full',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacious',
 'Spacio